In [2]:
import pandas as pd
import numpy as np
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("thecrowsnestapp-creds.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [3]:
from openskill.models import PlackettLuce, BradleyTerryFull
model = PlackettLuce(beta=25.0/30000.0)

In [ ]:
%load_ext scalene

In [4]:
targetElo = 1000
baseElo = 500
# baseSigma = baseElo // 3
# offset = baseElo * 2

In [5]:
class Sailor:
    def __init__(self, name,key, year, links, teams, seasons=[], skipperRank=0, crewRank=0,skipperRating=baseElo, crewRating=baseElo, races=[]):
        self.name = name
        self.key = key
        self.year = year
        self.links = links
        self.teams = teams
        self.skipperRank = skipperRank
        self.crewRank = crewRank
        self.seasons = seasons
        self.races = []
        self.rivals = {}
        # self.sr = model.rating(skipperRating, skipperRating / 3, name)
        # self.cr = model.rating(crewRating, crewRating / 3, name)
        self.sr = model.rating(name=name)
        self.cr = model.rating(name=name)
        self.avgSkipperRatio = 0
        self.avgCrewRatio = 0
        
    def __repr__(self):
        return f"{self.name}: {self.teams}, {str(self.sr.ordinal(target=targetElo, alpha=200 / model.sigma))} {str(self.cr.ordinal(target=targetElo, alpha=200 / model.sigma))} {self.seasons} {len(self.races)}"

In [5]:
p1 = Sailor("p1", teams=['nu'], links=[], key="p1", year="2027")
p2 = Sailor("p2", teams=['nu'], links=[], key="p2", year="2027")
p3 = Sailor("p3", teams=['nu'], links=[], key="p3", year="2027")
p4 = Sailor("p4", teams=['nu'], links=[], key="p4", year="2027")

team1 = [p1, p2]
team2 = [p3]

players = [p1,p2,p3,p4]

teams = [team1, team2]

ratings = [[p.sr] for p in players]
print(p1,p2,p3,p4)

ratings = [[p.sr for p in team] for team in teams]

ratings = model.rate(ratings, [2,1])
print(ratings)

for team, newRating in zip(teams, ratings):
    for i,player in enumerate(team):
        player.sr = newRating[i]

print(p1,p2,p3,p4)

p1: ['nu'], 1000.0 1000.0 [] 0 p2: ['nu'], 1000.0 1000.0 [] 0 p3: ['nu'], 1000.0 1000.0 [] 0 p4: ['nu'], 1000.0 1000.0 [] 0
[[PlackettLuceRating(mu=20.9120219532564, sigma=8.18761519144509), PlackettLuceRating(mu=20.9120219532564, sigma=8.18761519144509)], [PlackettLuceRating(mu=29.0879780467436, sigma=8.230685767392943)]]
p1: ['nu'], 912.3802330941071 1000.0 [] 0 p2: ['nu'], 912.3802330941071 1000.0 [] 0 p3: ['nu'], 1105.5020978695543 1000.0 [] 0 p4: ['nu'], 1000.0 1000.0 [] 0


In [6]:
def adjust_race_id(row):
    if row['Scoring'] == 'Combined':
        return row['raceID'][:-1]  # Remove the last character (A/B) for combined scoring
    return row['raceID']

In [7]:
def makePartnerKey(row):
    if row['PartnerLink'] != 'Unknown':
        return row['PartnerLink']
    if row ['Partner'] != "Unknown":
        return row['Partner'] + "/" + row['Team']
    return "Unknown"

In [8]:

df_races = pd.read_json("races_new_test.json")
# converters={"Teams": lambda x: [y.strip().split("'")[1] for y in x.strip("[]").split(", ")]}
df_races['raceNum'] = df_races['raceID'].apply(lambda id: int(id.split("/")[2][:-1]))  # Numeric part
df_races['raceDiv'] = df_races['raceID'].apply(lambda id: id.split("/")[2][-1])  # Division part (e.g., 'A', 'B')
df_races['adjusted_raceID'] = df_races.apply(adjust_race_id, axis=1) # to make combined division combined
df_races['Link'] = df_races['Link'].fillna('Unknown') # fill empty links
# df_races['key'] = np.where(df_races['Link'] == 'Unknown', df_races['Sailor'], df_races['Link'])
df_races['key'] = df_races.apply(
    lambda row: row['Sailor'] + "/" + row['Team'] if row['Link'] == 'Unknown' else row['Link'],
    axis=1
)
df_races['partnerKey'] = df_races.apply(
    makePartnerKey,
    axis=1
)
# df_races = df_races[df_races['Regatta'] == 'f24/seisa-women-fall'] # example combined regatta

# df_races = df_races.loc[df_races['raceID'].apply(lambda id: id.split("/")[0] == 'f24')]

df_races_full = df_races.sort_values(['Date', 'raceNum', 'raceDiv']).reset_index(drop=True)

df_races_skipper = df_races_full.loc[df_races_full['Position'].str.contains('Skipper')].sort_values(['Date', 'raceNum']).reset_index(drop=True)
df_races_crew = df_races_full.loc[df_races_full['Position'].str.contains('Crew')].sort_values(['Date', 'raceNum']).reset_index(drop=True) 
df_races

,Score,Div,Sailor,Link,GradYear,Position,Partner,PartnerLink,Venue,Regatta,Scoring,raceID,Date,Team,Teamlink,raceNum,raceDiv,adjusted_raceID,key,partnerKey
0,1.0,A,Campbell D'Eliscu,campbell-deliscu,19,Skipper,Meaghan MacRae,meaghan-macrae,Southern Cal,f16/rose-bowl-backup,2 Divisions,f16/rose-bowl-backup/1A,2017-01-07,Georgetown,/schools/georgetown/f16/,1,A,f16/rose-bowl-backup/1A,campbell-deliscu,meaghan-macrae
1,2.0,A,Campbell D'Eliscu,campbell-deliscu,19,Skipper,Meaghan MacRae,meaghan-macrae,Southern Cal,f16/rose-bowl-backup,2 Divisions,f16/rose-bowl-backup/2A,2017-01-07,Georgetown,/schools/georgetown/f16/,2,A,f16/rose-bowl-backup/2A,campbell-deliscu,meaghan-macrae
2,1.0,A,Campbell D'Eliscu,campbell-deliscu,19,Skipper,Meaghan MacRae,meaghan-macrae,Southern Cal,f16/rose-bowl-backup,2 Divisions,f16/rose-bowl-backup/3A,2017-01-07,Georgetown,/schools/georgetown/f16/,3,A,f16/rose-bowl-backup/3A,campbell-deliscu,meaghan-macrae
3,3.0,B,Sean Segerblom,sean-segerblom,20,Skipper,Claire Mohun,claire-mohun,Southern Cal,f16/rose-bowl-backup,2 Divisions,f16/rose-bowl-backup/1B,2017-01-07,Georgetown,/schools/georgetown/f16/,1,B,f16/rose-bowl-backup/1B,sean-segerblom,claire-mohun
4,16.0,B,Sean Segerblom,sean-segerblom,20,Skipper,Claire Mohun,claire-mohun,Southern Cal,f16/rose-bowl-backup,2 Divisions,f16/rose-bowl-backup/2B,2017-01-07,Georgetown,/schools/georgetown/f16/,2,B,f16/rose-bowl-backup/2B,sean-segerblom,claire-mohun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788903,14.0,B,Chloe Lighterink,chloe-lighterink,24,Crew,Claire Desbaillets,None,Hawaii,s24/peter-wenner-rainbow-invite,2 Divisions,s24/peter-wenner-rainbow-invite/7B,2024-01-20,UC Davis,/schools/uc-davis/s24/,7,B,s24/peter-wenner-rainbow-invite/7B,chloe-lighterink,None
788904,16.0,B,Chloe Lighterink,chloe-lighterink,24,Crew,Claire Desbaillets,None,Hawaii,s24/peter-wenner-rainbow-invite,2 Divisions,s24/peter-wenner-rainbow-invite/8B,2024-01-20,UC Davis,/schools/uc-davis/s24/,8,B,s24/peter-wenner-rainbow-invite/8B,chloe-lighterink,None
788905,16.0,B,Chloe Lighterink,chloe-lighterink,24,Crew,Claire Desbaillets,None,Hawaii,s24/peter-wenner-rainbow-invite,2 Divisions,s24/peter-wenner-rainbow-invite/9B,2024-01-20,UC Davis,/schools/uc-davis/s24/,9,B,s24/peter-wenner-rainbow-invite/9B,chloe-lighterink,None
788906,16.0,B,Chloe Lighterink,chloe-lighterink,24,Crew,Claire Desbaillets,None,Hawaii,s24/peter-wenner-rainbow-invite,2 Divisions,s24/peter-wenner-rainbow-invite/10B,2024-01-20,UC Davis,/schools/uc-davis/s24/,10,B,s24/peter-wenner-rainbow-invite/10B,chloe-lighterink,None


In [9]:
teamRegions = {'Hawaii': 'PCCSC', 'Brown': 'NEISA', 'Southern Cal': 'PCCSC', 'Salve Regina': 'NEISA', 'UC Santa Barbara': 'PCCSC', 'Cal Poly': 'PCCSC', 'Washington': 'NWICSA', 'Channel Islands': 'PCCSC', 'UC San Diego': 'PCCSC', 'British Columbia': 'NWICSA', 'UC Los Angeles': 'PCCSC', 'Westmont College': 'PCCSC', 'Arizona State': 'PCCSC', 'Texas A&M Galveston': 'SEISA', 'Texas A&M': 'SEISA', 'Tulane': 'SEISA', 'Rice': 'SEISA', 'Texas': 'SEISA', 'Oklahoma State': 'SEISA', 'Texas A&M C. Christ': 'SEISA', 'Central Oklahoma': 'SEISA', 'Notre Dame': 'MCSA', 'Jacksonville': 'SAISA', 'Florida': 'SAISA', 'Tennessee': 'SAISA', 'Rollins': 'SAISA', 'North Carolina State': 'SAISA', 'Georgia Tech': 'SAISA', 'Auburn': 'SAISA', 'Charleston': 'SAISA', 'South Florida': 'SAISA', 'Old Dominion': 'MAISA', 'Eckerd': 'SAISA', 'Florida State': 'SAISA', 'U. Miami': 'SAISA', 'UW Milwaukee': 'MCSA', 'Stony Brook': 'MAISA', 'Duke': 'SAISA', 'Clemson': 'SAISA', 'U South Carolina': 'SAISA', 'UNC Wilmington': 'SAISA', 'Georgia': 'SAISA', 'Berkeley': 'PCCSC', 'CSU Long Beach': 'PCCSC', 'Monterey Bay': 'PCCSC', 'UC Irvine': 'PCCSC', 'UC Davis': 'PCCSC', 'Rhode Island': 'NEISA', 'Georgetown': 'MAISA', 'Dartmouth': 'NEISA', 'MIT': 'NEISA', 'George Washington': 'MAISA', 'Navy': 'MAISA', 'Fordham': 'MAISA', 'Northeastern': 'NEISA', 'Christopher Newport': 'MAISA', 'Victoria': 'NWICSA', 'Boston University': 'NEISA', 'Miami University': 'MCSA', 'Hampton': 'MAISA', 'Virginia': 'MAISA', 'Stevens': 'MAISA', 'Columbia': 'MAISA', 'NY Maritime': 'MAISA', 'Kings Point': 'MAISA', "St. Mary's": 'MAISA', 'Maryland': 'MAISA', 'Virginia Tech': 'MAISA', 'Drexel': 'MAISA', 'Maryland/Baltimore': 'MAISA', 'Buffalo': 'MAISA', 'UC Santa Cruz': 'PCCSC', 'Santa Clara': 'PCCSC', 'Wisconsin': 'MCSA', 'Michigan': 'MCSA', 'Washington College': 'MAISA', 'Minnesota': 'MCSA', 'Yale': 'NEISA', 'Hobart & William': 'MAISA', 'Vermont': 'NEISA', 'Connecticut College': 'NEISA', 'Harvard': 'NEISA', 'Roger Williams': 'NEISA', 'Syracuse': 'MAISA', 'Tufts': 'NEISA', 'Middlebury': 'NEISA', 'New College': 'SAISA', 'William and Mary': 'MAISA', 'Gannon': 'MAISA', 'Boston College': 'NEISA', 'Stanford': 'PCCSC', 'Bowdoin': 'NEISA', 'Lewis & Clark': 'NWICSA', 'Monmouth': 'MAISA', 'American': 'MAISA', 'Michigan State': 'MCSA', 'Hope': 'MCSA', 'Western Michigan': 'MCSA', 'Toledo': 'MCSA', 'Ohio State': 'MCSA', 'Mass Maritime': 'NEISA', 'Coast Guard': 'NEISA', 'Bates': 'NEISA', 'Fairfield': 'NEISA', 'Sacred Heart': 'NEISA', 'Wentworth Institute': 'NEISA', 'Providence': 'NEISA', 'Iowa State': 'MCSA', 'Iowa': 'MCSA', 'Indiana': 'MCSA', 'Davidson': 'SAISA', 'Oregon State': 'NWICSA', 'Western Washington': 'NWICSA', 'U. Rochester': 'MAISA', 'Army': 'MAISA', 'New Hampshire': 'NEISA', 'U. Connecticut': 'NEISA', 'UMass Dartmouth': 'NEISA', 'Wesleyan': 'NEISA', 'U. Mass/ Amherst': 'NEISA', 'U New England': 'NEISA', 'Denison': 'MCSA', 'Northern Michigan': 'MCSA', 'Ohio': 'MCSA', 'Pennsylvania': 'MAISA', 'Villanova': 'MAISA', 'Maine Maritime': 'NEISA', 'Michigan Tech': 'MCSA', 'Illinois': 'MCSA', 'Chicago': 'MCSA', 'Northwestern': 'MCSA', 'Grand Valley State': 'MCSA', 'Washington U': 'MCSA', 'Marquette': 'MCSA', 'Lake Forest': 'MCSA', 'Cornell': 'MAISA', 'Oregon': 'NWICSA', 'Portland State': 'NWICSA', 'Princeton': 'MAISA', "Queen's": 'MAISA', 'Penn State': 'MAISA', 'Ocean County': 'MAISA', 'Delaware': 'MAISA', 'Rutgers': 'MAISA', 'Worcester Polytech': 'NEISA', 'Emmanuel College': 'NEISA', "St. John's": 'MAISA', 'U Pittsburgh': 'MAISA', 'Webb Institute': 'MAISA', 'McGill': 'NEISA', 'Citadel': 'SAISA', 'Colgate': 'MAISA', 'Catholic U America': 'MAISA', 'Loyola College': 'MAISA', 'Ottawa': 'MAISA', 'Royal Military': 'MAISA', 'Dalhousie': 'NEISA', 'U Toronto': 'MAISA', 'New Orleans': 'SEISA', 'Kansas': 'SEISA', 'Bentley': 'NEISA', 'Brandeis': 'NEISA', 'Cal Maritime': 'PCCSC', 'San Diego State': 'PCCSC', 'Loyola': 'SEISA', 'North Texas': 'SEISA', 'Vanderbilt': 'SAISA', 'Purdue': 'MCSA', 'North Carolina': 'SAISA', 'Hillsdale': 'MCSA', 'Amherst': 'NEISA', 'Williams': 'NEISA', 'Hamilton': 'MAISA', 'Rochester': 'MAISA', 'Wellesley': 'NEISA', 'Hosei Univerisity': 'GUEST', 'Colorado': 'SEISA', 'John Carroll': 'MCSA', 'U.  Mass/ Boston': 'NEISA', 'Mercyhurst': 'MAISA', 'Penn State Behrend': 'MAISA', 'Indiana U Pennsylvan': 'MAISA', 'U Nebraska': 'MCSA', 'U Maine': 'NEISA', 'Texas Christian': 'SEISA', 'Embry-Riddle': 'SAISA', 'Palm Beach Atlantic': 'SAISA', 'U of Central Florida': 'SAISA', 'Baldwin-Wallace': 'MCSA', "Saint Mary's College": 'MCSA', 'Olin': 'NEISA', 'Baylor': 'SEISA', 'Texas Tech': 'SEISA', 'Wake Forest': 'SAISA', 'Georgia Southern': 'SAISA', 'East Carolina': 'SAISA', 'Florida Tech': 'SAISA', 'Saint Thomas': 'MCSA', 'Cincinnati': 'MCSA', 'Florida Gulf Coast': 'SAISA', 'Saginaw Valley': 'MCSA', 'Coastal Georgia': 'SAISA', 'Cleveland State': 'MCSA', 'Sewanee': 'SAISA', 'Case Western': 'MCSA', 'Oklahoma': 'SEISA', 'Gonzaga': 'PCCSC'}


In [10]:
merges = [{'first': 'carter-anderson', 'second': 'carter-anderson-2027'}, {'first': 'elliott-bates', 'second':'elliott-bates-2021'}]

In [11]:
# Function to add a sailor to the dictionary
def add_sailor(group,names_group, links_group, seasons_group,teams_group, years_group, people):
    """Summary

    Args:
        group (pandas group): The grouped list of sailor keys and teams
        names_group (pandas group): The list of names grouped by sailor key
        links_group (pandas group): The list of links grouped by sailor key
        seasons_group (pandas group): The list of seasons grouped by sailor key
        teams_group (pandas group): The list of teams grouped by sailor key
        years_group (pandas group): The list of years grouped by sailor key
        people (dict): The people dictionary to be added to
    """

    for key, teams in group.items():
        if key not in people.keys():
            # If no teams are associated, set "Unknown"
            teams = teams if len(teams) > 0 else ["Unknown"]
            
            # Retrieve the precomputed values
            name = names_group.get(key,[])[0]
            link = links_group.get(key,[])
            seasons = seasons_group.get(key,[])
            teams = teams_group.get(key,[])
            year = years_group.get(key, [])[0]
            
            # Add the sailor to the people dictionary
            people[key] = Sailor(name, key, year, list([link]), teams, {'skipper': seasons.get('Skipper', []), 'crew': seasons.get('Crew', [])})
            
def setupPeople():
    """Generates a dictionary with all of the sailors based on the df_races_full dataframe

    Returns:
        dict: The filled out dictionary of people
    """
    
    people = {}
    
    try:
        df_s = pd.read_json("sailorsasf.json")
    except:
        df_s = pd.DataFrame(columns=['Sailor'])

    # create sailors from file (NOT WORKING)
    for sailor in list(df_s['Sailor'].unique()):
        # print(sailor)
        positions = df_s.loc[df_s['Sailor'] == sailor, 'Pos']
        for pos in positions:
            teams = df_s.loc[(df_s['Sailor'] == sailor)& (df_s['Pos'] == pos), 'Teams'].iat[0]
            seasons = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Seasons'].iat[0]
            year = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'GradYear'].iat[0]
            link = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Link'].iat[0]
            rating = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Elo'].iat[0]
            rank = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Rank'].iat[0]
            races = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Races'].iat[0]
            people[link] = Sailor(sailor, year, link, teams, pos, seasons, rank, rating, races)
    
    # Do merges if necessary (merging two techscore links)
    # We must merge here before the calculation is done, because each new rating will need the accurate history
    for merge in merges:
        if merge['second'] in people.keys():
            people[merge['first']].links.append(people[merge['second']])
            del people[merge['second']]
        df_races_full['key'] = df_races_full['key'].replace(merge['second'], merge['first'])
        df_races_skipper['key'] = df_races_skipper['key'].replace(merge['second'], merge['first'])
        df_races_crew['key'] = df_races_crew['key'].replace(merge['second'], merge['first'])

    # Pre-group the data for skippers and crews
    grouped = df_races_full.groupby(['key'])['Team'].unique()
    
    # Precompute seasons for skippers and crew
    names = (
        df_races_full.assign(Season=df_races_full['raceID'].str.split('/').str[0])
        .groupby('key')['Sailor']
        .unique()
    )
    links = (
        df_races_full.assign(Season=df_races_full['raceID'].str.split('/').str[0])
        .groupby('key')['Link']
        .unique()
    )
    seasons = (
        df_races_full.assign(Season=df_races_full['raceID'].str.split('/').str[0])
        .groupby(['key', 'Position'])['Season']
        .unique()
    )
    teams = (
        df_races_full.assign(Season=df_races_full['raceID'].str.split('/').str[0])
        .groupby('key')['Team']
        .unique()
    )

    years = (
        df_races_full.assign(Season=df_races_full['raceID'].str.split('/').str[0])
        .groupby('key')['GradYear']
        .unique()
    )
    
    # Add all sailors to the people dictionary
    add_sailor(grouped, names, links, seasons, teams, years, people)
    
    return people

In [12]:
# %%scalene # for profiling the code

# Set up people dictionary
people = setupPeople()
# Pre calculate the number of races to rate
leng = len(df_races['raceID'].unique())

# List of residuals (errors)
residuals = []

# Current race count for print statement
i = 0

# Creates pandas group object
grouped = df_races_full.groupby(['Date', 'Regatta', 'adjusted_raceID'], sort=False)

# Iterate through each race
for (date, regatta, race), row in grouped:
    # Calculate for each position
    
    # Print status every 1000 to help with performance and output length
    if i % 1000 == 0:
        print(f"Currently analyzing race {i}/{leng} Regatta:{regatta}, Date:{date}")
    i += 1
    
    # Filter by current position 
    scores = row[row['Position'] == 'Skipper'] 
    # Grab lists for each datapoint
    keys = scores['key'] # the sailor keys
    names = scores['Sailor'] # the sailor names
    scoreVals = list(scores['Score']) # the score values
    
    partners = list(scores['Partner'])
    partnerKeys = list(scores['partnerKey'])
    
    # check for invalid race conditions
    if len(keys) < 2: # less than two sailors
        continue
    if np.isnan(scoreVals[0]): # B division did not complete the set
        continue
    
    # Grab people objects 
    skippers = [people[key] for key in keys]
    crews = [people[key] if key != 'Unknown' and key is not None and key in people.keys() else None for key in partnerKeys]
    
    # for p in skippers:
    #     if len(p.races) > 0:
    #         days = (date - sorted(p.races, key=lambda r: r['date'])[-1]['date']).days
            
    #         if p.sr.sigma < model.sigma:
    #             # number of weeks since last competition, scaled by amount it would take to get back to default if a player skipped a season
    #             # should be 0 if they competed last week
    #             p.sr.sigma += ((days // 7) - 1) * (model.sigma / 10)
    #             if p.sr.sigma > model.sigma:
    #                 p.sr.sigma = model.sigma
    # for p in skippers:
    #     if len(p.races) > 0:
    #         days = (date - sorted(p.races, key=lambda r: r['date'])[-1]['date']).days
    #         if p.sr.sigma < model.sigma:
    #             # number of weeks since last competition, scaled by amount it would take to get back to default if a player skipped a season
    #             # should be 0 if they competed last week
    #             p.sr.sigma += ((days // 7) - 1) * (model.sigma / 10)
    #             if p.sr.sigma > model.sigma:
    #                 p.sr.sigma = model.sigma
    
    # Seperate out the openskill rating objects for use in the model
    ratings = [[r.sr, p.cr if p is not None else None] for r,p in zip(skippers,crews)]
    ratings = [[r for r in rating if r is not None] for rating in ratings]
    
    # grab starting rating values for change calculation later
    startingRating = [[r.ordinal(target=targetElo, alpha=200 / model.sigma) for r in team] for team in ratings]
    
    # Calculate regatta average
    regattaAvg = sum([sum(el) for el in startingRating]) / (len(skippers) + len(crews))
    
    # Make predictions
    predictions = model.predict_rank(ratings)
    
    weights = [[0, -0.5] if len(r) > 1 else [0] for r in ratings]
    # print(weights)
    
    # Rate using the model
    ratings = model.rate(ratings, scoreVals, tau=0, weights=weights)
    
    # calculate error and add to list (residuals)
    for pred, score, racer in zip(predictions, scoreVals, skippers):
        residuals.append(score - pred[0])

    # Update racers' ratings
    for skipper, crew, new_rating in zip(skippers, crews, ratings):
        skipper.sr = new_rating[0]
        # if (len(skipper.races) < 20):
        # skipper.sr.sigma -= 3 * np.exp(-0.1 * (skipper.sr.sigma - 1))
        # skipper.sr.sigma = max(skipper.sr.sigma, 1)
        if len (new_rating) > 1:
            crew.cr = new_rating[1]
            # if (len(crew.races) < 20):
            # crew.cr.sigma -= 3 * np.exp(-0.1 * (crew.cr.sigma - 1))
            # crew.cr.sigma = max(crew.cr.sigma, 1)

    # Pre-calculate lists for sailor's race values
    changes = [[r.ordinal(target=targetElo, alpha=200 / model.sigma) - startingRating[i][j] for j,r in enumerate(team)] for i,team in enumerate(ratings)]
    
    # Common values for each sailor
    venue = scores['Venue'].iat[0]
    scoring = scores['Scoring'].iat[0]
    actualID = scores['raceID'].iat[0]
    
    # Make list of regions and combine PCCSC and NWICSA (those shouldnt count as cross regional for rating purposes)
    regions = [teamRegions[p.teams[-1]] if p.teams[-1] in teamRegions.keys() else None for p in skippers]
    regions = ['PCCSC' if reg == 'NWICSA' else reg for reg in regions]
    
    # Check if race has any out of region sailors
    isCross = 1 if len(set(regions)) > 1 else 0
    
    # Only calculate number of cross regional sailors if it is the current season
    doCr = race.split("/")[0] == 'f24' and isCross == 1
    
    # Loop through each sailor and the associated values
    for skipper, score, pred, change, partner, partnerKey, crew in zip(skippers, scoreVals, predictions, changes, partners, partnerKeys, crews):
        outLinks = 0
        
        if(isCross == 1):
            # Calculate the number of sailors that are not in the sailor's region
            outLinks = sum(1 for reg in regions # adds 1 each time that a region in the regatta ... 
                            if reg is not None # Double check that the region is not none # first double check that the sailor's team is in the list of regions
                            and skipper.teams[-1] in teamRegions.keys()
                            and ('PCCSC' if reg == 'NWICSA' else reg) != ('PCCSC' if teamRegions[skipper.teams[-1]] == 'NWICSA' else teamRegions[skipper.teams[-1]])) # The sailor's region is not the same as the opponent)
            # Note: We don't need to filter out the sailor themselves from this list, because they will have the same region as themseleves so it will not be counted.
        
        # add race to each sailor's score
        skipper.races.append({
            'score': int(score), # Need to rewrite to include DNF and such (correctly evaluating score but its hard to tell )
            'pos': 'Skipper',
            'predicted': pred[0],
            'ratio': 1 - ((int(score) - 1) / (len(skippers) - 1)), # Calculate ratio here
            'change': change[0],
            'regAvg': regattaAvg,
            'cross': isCross,
            'outLinks': outLinks,
            'skipperRating': skipper.sr.ordinal(target=targetElo, alpha=200 / model.sigma), # add offset to prevent negative ratings
            'skipperSigma': skipper.sr.sigma,
            'crewRating': skipper.cr.ordinal(target=targetElo, alpha=200 / model.sigma), # add offset to prevent negative ratings
            'date' :date,
            'partner': {'name': partner, 'key': partnerKey},
            'venue': venue,
            'raceID': actualID,
            'scoring': scoring
        })
        
        if crew is not None:
            # Append to crew's races as well
            crew.races.append({
                'score': int(score), # Need to rewrite to include DNF and such (correctly evaluating score but its hard to tell )
                'pos': 'Crew',
                'predicted': pred[0],
                'ratio': 1 - ((int(score) - 1) / (len(skippers) - 1)), # Calculate ratio here
                'change': change[1],
                'regAvg': regattaAvg,
                'cross': isCross,
                'outLinks': outLinks,
                'skipperRating': crew.sr.ordinal(target=targetElo, alpha=200 / model.sigma), # add offset to prevent negative ratings
                'skipperSigma': skipper.sr.sigma,
                'crewRating': crew.cr.ordinal(target=targetElo, alpha=200 / model.sigma), # add offset to prevent negative ratings
                'date' :date,
                'partner': {'name': skipper.name, 'key': skipper.key},
                'venue': venue,
                'raceID': actualID,
                'scoring': scoring
            })

# Calculate statiscs about the accuracy of the model. (Lower is better)
me = np.array(residuals).mean()
mse = (np.array(residuals) ** 2).mean()
print(me, mse)

Currently analyzing race 0/32581 Regatta:s16/peter-wenner-rainbow-invite, Date:2016-01-16 00:00:00
Currently analyzing race 1000/32581 Regatta:s16/oberg, Date:2016-04-16 00:00:00
Currently analyzing race 2000/32581 Regatta:f16/fj-invitational, Date:2016-09-11 00:00:00
Currently analyzing race 3000/32581 Regatta:f16/david-lee-arnoff, Date:2016-10-08 00:00:00
Currently analyzing race 4000/32581 Regatta:s17/hawkeye-invitational, Date:2017-03-25 00:00:00
Currently analyzing race 5000/32581 Regatta:s17/army-spring-open, Date:2017-04-29 00:00:00
Currently analyzing race 6000/32581 Regatta:f17/saisa-women-fall, Date:2017-09-30 00:00:00
Currently analyzing race 7000/32581 Regatta:f17/cedarfest, Date:2017-10-28 00:00:00
Currently analyzing race 8000/32581 Regatta:s18/saisa-co-ed-champs, Date:2018-04-07 00:00:00
Currently analyzing race 9000/32581 Regatta:f18/jack-boehringer-52, Date:2018-09-08 00:00:00
Currently analyzing race 10000/32581 Regatta:f18/harvard-coed-showcase, Date:2018-10-06 00:00

In [ ]:
# Compute season rivals
grouped = df_races_full.groupby('adjusted_raceID') # [df_races_full['raceID'].str.startswith('f24')]

i = 0
leng = len(grouped)
for raceID, scores in grouped:
    # Report status every 100 races
    if i % 1000 == 0:
        print(f"Currently analyzing race {i}/{leng} Regatta:{raceID}")
    i += 1
    
    season = raceID.split("/")[0]
    
    sailor_scores = scores.set_index('key')[['Position', 'Score', 'Team', 'Sailor']]

    for sailor, sailor_data in sailor_scores.iterrows():
        pos = sailor_data['Position']
        score = sailor_data['Score']
        p = people[sailor]
        
        others = sailor_scores.iterrows()

        for other_key, other_sailor_data in others:
            if other_key != sailor:
                other_pos = other_sailor_data['Position']
                if pos == other_pos:
                    other_score = other_sailor_data['Score']
                    other_team = other_sailor_data['Team']
                    other_name = other_sailor_data['Sailor']
                    
                    if pos not in p.rivals:
                        p.rivals[pos] = {}
                    
                    if other_key not in p.rivals[pos]:
                        p.rivals[pos][other_key] = {'name': other_name,'races': {}, 'team': other_team, 'wins': {}}
                        
                    if season not in p.rivals[pos][other_key]['races'].keys():
                        p.rivals[pos][other_key]['races'][season] = 0
                    if season not in p.rivals[pos][other_key]['wins'].keys():
                        p.rivals[pos][other_key]['wins'][season] = 0
                    
                    p.rivals[pos][other_key]['races'][season] += 1
                    if other_score > score:
                        p.rivals[pos][other_key]['wins'][season] += 1

In [ ]:
# Filter sailors who have 'f24' in their seasons list
eligible_skippers = [p for p in people.values()
                    if 'f24' in p.seasons['skipper'] 
                    and sum([race['outLinks'] for race in p.races]) > 70]

eligible_crews = [p for p in people.values()
                    if 'f24' in p.seasons['crew']
                    and sum([race['outLinks'] for race in p.races]) > 70]

for p in people.values():
    p.skipperRank = 0
    p.crewRank = 0

for i,s in enumerate(sorted([p for p in eligible_skippers], key=lambda p: p.sr.ordinal(), reverse=True)):
    s.skipperRank = i + 1
for i,s in enumerate(sorted([p for p in eligible_crews], key=lambda p: p.cr.ordinal(), reverse=True)):
    s.crewRank = i + 1

allRows = []
for sailor,p in people.items():
    avgSkipperRatio = float(np.array([r['ratio'] for r in p.races if r['pos'] == 'Skipper']).mean())
    avgCrewRatio = float(np.array([r['ratio'] for r in p.races if r['pos'] == 'Crew']).mean())
    p.avgSkipperRatio = avgSkipperRatio
    p.avgCrewRatio = avgCrewRatio
    allRows.append([p.name, sailor, p.skipperRank,p.crewRank, p.teams, p.sr.ordinal(target=targetElo, alpha=200 / model.sigma), p.cr.ordinal(target=targetElo, alpha=200 / model.sigma), sum([race['outLinks'] for race in p.races]), p.year, p.links, p.sr.mu, p.cr.mu, avgSkipperRatio,avgCrewRatio, p.sr.sigma, p.cr.sigma, p.seasons, sum([race['cross'] for race in p.races]), p.races, p.rivals])
    
df_sailors = pd.DataFrame(allRows, columns=['Sailor', 'key', 'SkipperRank', 'CrewRank', 'Teams', 'SkipperOrdinal', 'CrewOrdinal','outLinks','GradYear', 'Links', 'SkipperMU','CrewMU', 'skipperAvgRatio', 'crewAvgRatio', 'SkipperSigma','CrewSigma', 'Seasons', 'Cross', 'Races', 'Rivals'])

# df_sailors.to_json('sailorsexperiment19.json', index=False)
df_sailors.head()

/var/folders/10/cpsfwdm906d0_fmx9wrl17gh0000gn/T/ipykernel_12803/3655594634.py:22: RuntimeWarning: Mean of empty slice.
  avgCrewRatio = float(np.array([r['ratio'] for r in p.races if r['pos'] == 'Crew']).mean())
/Users/carter/.pyenv/versions/3.13.1/lib/python3.13/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/10/cpsfwdm906d0_fmx9wrl17gh0000gn/T/ipykernel_12803/3655594634.py:21: RuntimeWarning: Mean of empty slice.
  avgSkipperRatio = float(np.array([r['ratio'] for r in p.races if r['pos'] == 'Skipper']).mean())


,Sailor,key,SkipperRank,CrewRank,Teams,SkipperOrdinal,CrewOrdinal,outLinks,GradYear,Links,SkipperMU,CrewMU,skipperAvgRatio,crewAvgRatio,SkipperSigma,CrewSigma,Seasons,Cross,Races,Rivals
0,A.J. Crane,A.J. Crane/Tufts,0,0,[Tufts],1108.356165,1000.0,0,04 *,[[Unknown]],28.957147,25.0,0.488095,NaN,8.147436,8.333333,"{'skipper': ['s23'], 'crew': []}",0,"[{'score': 10, 'pos': 'Skipper', 'predicted': ...",{}
1,Aaron Babier,Aaron Babier/U Toronto,0,0,[U Toronto],1000.000000,1000.0,0,17 *,[[Unknown]],25.000000,25.0,NaN,NaN,8.333333,8.333333,"{'skipper': [], 'crew': ['f16']}",0,[],{}
2,Aaron Heard,Aaron Heard/Oregon State,0,0,[Oregon State],1626.330872,1000.0,0,25 *,[[Unknown]],48.581422,25.0,0.279270,NaN,7.494767,8.333333,"{'skipper': ['s23', 'f23'], 'crew': ['f22', 's...",0,"[{'score': 7, 'pos': 'Skipper', 'predicted': 6...",{}
3,Aaron Klein,Aaron Klein/Tufts,0,0,[Tufts],1136.752575,1000.0,0,18 *,[[Unknown]],29.257335,25.0,0.397436,NaN,7.853104,8.333333,"{'skipper': ['s23'], 'crew': []}",0,"[{'score': 9, 'pos': 'Skipper', 'predicted': 2...",{}
4,Aaron Le Roy,Aaron Le Roy/Marquette,0,0,[Marquette],1159.606909,1000.0,0,24 *,[[Unknown]],30.115849,25.0,0.083333,NaN,7.821854,8.333333,"{'skipper': ['s22'], 'crew': []}",0,"[{'score': 4, 'pos': 'Skipper', 'predicted': 3...",{}


In [17]:
df_temp = pd.read_json("races_new_test.json")
df_temp

,Score,Div,Sailor,Link,GradYear,Position,Partner,PartnerLink,Venue,Regatta,Scoring,raceID,Date,Team,Teamlink
0,1.0,A,Nicholas Mueller,nicholas-mueller,27,Skipper,Mateo Krok,mateo-krok,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/1A,2024-03-09,Cal Maritime,/schools/cal-maritime/s24/
1,2.0,A,Nicholas Mueller,nicholas-mueller,27,Skipper,Mateo Krok,mateo-krok,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/2A,2024-03-09,Cal Maritime,/schools/cal-maritime/s24/
2,1.0,A,Nicholas Mueller,nicholas-mueller,27,Skipper,Mateo Krok,mateo-krok,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/3A,2024-03-09,Cal Maritime,/schools/cal-maritime/s24/
3,3.0,A,Nicholas Mueller,nicholas-mueller,27,Skipper,Mateo Krok,mateo-krok,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/4A,2024-03-09,Cal Maritime,/schools/cal-maritime/s24/
4,1.0,A,Nicholas Mueller,nicholas-mueller,27,Skipper,Mateo Krok,mateo-krok,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/5A,2024-03-09,Cal Maritime,/schools/cal-maritime/s24/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,1.0,A,Hayley Lai,None,24 *,Crew,Enzo Cremers,enzo-cremers,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/4A,2024-03-09,Berkeley,/schools/berkeley/s24/
204,1.0,B,Jake Cole,None,25 *,Crew,Victoria Chen,victoria-chen,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/1B,2024-03-09,Berkeley,/schools/berkeley/s24/
205,4.0,B,Jake Cole,None,25 *,Crew,Victoria Chen,victoria-chen,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/2B,2024-03-09,Berkeley,/schools/berkeley/s24/
206,8.0,B,Jake Cole,None,25 *,Crew,Victoria Chen,victoria-chen,UC Santa Cruz,s24/st-francis-invite,2 Divisions,s24/st-francis-invite/3B,2024-03-09,Berkeley,/schools/berkeley/s24/


In [ ]:
win_ratios = []

for rival, stats in people['carter-anderson'].rivals['Skipper'].items():
    races = stats['races']
    wins = stats['wins']
    team = stats['team']
    win_ratio = wins / races if races > 0 else 0  # Handle division by zero
    win_ratios.append((rival, races, win_ratio, team))

# Sort the list by win_ratio in descending order
win_ratios_sorted = sorted(win_ratios, key=lambda x: x[1], reverse=True)

# Print the sorted win ratios
for rival, races, win_ratio, team in win_ratios_sorted:
    print(f"{rival}, Team: {team}, Races: {races}, Win Percentage: {win_ratio * 100:.0f}%")

In [ ]:
# Updates only changed sailors
col = db.collection('sailorsElo')

for doc in col.limit(10).stream():
    doc_id = doc.id
    data = doc.to_dict()
    print(doc_id)
    changes = {}
    person = people[data['Name'] + "/" + data['Position']]
    if len(person.races) < 1:
        print("no races found for sailor, skipping...")
        continue
    
    print(data['Name'])
    try:
        data['Link']
    except:
        changes['Link'] = list(person.link)[0]
        # print("No link!")
    try:
        data['Year']
    except:
        changes['Year'] = int(list(person.year)[0][:2])
        # print("No year!")
    
    # print(person.races[0])
    # check races
    changes['races'] = firestore.ArrayUnion(person.races)
    # for raceID in [r['raceID'] for r in person.races]:
    #     if raceID not in [r['raceID'] for r in data['races']]:
    #         changes['races'].append(raceID)
    #         print("needs update!", data['races'])
    #         print(person.races)
    print(changes)
    col.document(doc_id).update(changes)

In [55]:
# Initialize Firestore client
col = db.collection('sailorsElo')

# Initialize the batch
batch = db.batch()

# Number of documents to commit in each batch
batch_size = 40

# Iterate over the people values
# for i, p in enumerate(people.values()):
p = people['justin-callahan']
# Prepare the document data to be written
doc_data = {
    "Name": p.name,
    "key": p.key,
    "Teams": p.teams.tolist() if isinstance(p.teams, np.ndarray) else p.teams,
    "SkipperRating": int(p.sr.ordinal(target=targetElo, alpha=200 / model.sigma)),
    "CrewRating": int(p.cr.ordinal(target=targetElo, alpha=200 / model.sigma)),
    "SkipperRank": int(p.skipperRank),
    "CrewRank": int(p.crewRank),
    "Links": p.links.tolist() if isinstance(p.links, np.ndarray) else p.links if isinstance(p.links, str) else p.links[0].tolist(),
    "Year": p.year,
    "Seasons": {'skipper': list(p.seasons['skipper']), 'crew': list(p.seasons['crew'])},
    "Cross":  sum([race['cross'] for race in p.races]),
    "OutLinks": sum([race['outLinks'] for race in p.races]),
    "races": [{'sailor': p.name,
                'key': p.key,
                'pos': race['pos'],
            "raceID": race['raceID'], 
            "score": float(race['score']), 
            "predicted": int(race['predicted']), 
            "change": float(race['change']), 
            'regAvg': float(race['regAvg']), 
            'skipperRating': float(race['skipperRating']),
            'skipperSigma': float(race['skipperSigma']),
            'crewRating': float(race['crewRating']),
            'date': race['date'],
            'partner': race['partner'],
            'ratio': float(race['ratio']),
            'venue': race['venue'],
            'scoring': race['scoring']
            } for race in p.races],
    "Rivals": p.rivals
}

# Add the set operation to the batch
doc_ref = col.document('OWmiJiRa0KkqlfJStPsK')  # This creates a new document with an auto-generated ID
batch.set(doc_ref, doc_data)

# Commit the batch every 20 documents
# if (i + 1) % batch_size == 0:
#     batch.commit()
#     batch = db.batch()  # Start a new batch for the next set of documents

# Commit any remaining operations if there are less than 20 documents left
if (i + 1) % batch_size != 0:
    batch.commit()

In [37]:
# %%scalene
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

def getCounts(races):
    season_counts = defaultdict(int)
    
    for race in races:
        season = race["raceID"].split("/")[0]
        season_counts[season] += 1

    return dict(season_counts)

# batch = db.batch()
col = db.collection('eloTeams')
teams = []
scrape = False
teamNames = teamRegions.keys()
lenteams = len(teamNames)

season_mask = df_sailors['Seasons'].apply(lambda x: 'f24' in x['skipper'] or 'f24' in x['crew'])

# Explode the Teams column to enable grouping
df_exploded = df_sailors.loc[season_mask].explode('Teams')

# Group by team and compute necessary aggregates
team_stats = df_exploded.groupby('Teams').agg(
    numCurMembers=('Teams', 'count'),
    avgSkipperOrdinal=('SkipperOrdinal', 'mean'),
    avgCrewOrdinal=('CrewOrdinal', 'mean'),
    avgSkipperRatio=('skipperAvgRatio', 'mean'),
    avgCrewRatio=('crewAvgRatio', 'mean')
)

# Calculate the average values as in the original code
team_stats['avg'] = (team_stats['avgSkipperOrdinal'] + team_stats['avgCrewOrdinal']) / 2
team_stats['avgRatio'] = (team_stats['avgSkipperRatio'] + team_stats['avgCrewRatio']) / 2

team_stats = team_stats.reindex(teamNames, fill_value=0)


team_link_map = df_races.drop_duplicates('Team').set_index('Team')['Teamlink'].to_dict()

# Optional: Loop for printing (if necessary)
for i, (team, row) in enumerate(team_stats.iterrows()):
    print(f"{i}/{len(team_stats)} {team}")
    avg = row['avg']
    avgRatio = row['avgRatio']
    numCurMembers = row['numCurMembers']

# for i,team in enumerate(teamNames):
#     print(f"{i}/{lenteams} {team}")
#     temp = df_sailors.loc[(df_sailors['Teams'].apply(lambda x: team in x)) & (df_sailors['Seasons'].apply(lambda x: 'f24' in x['skipper'] or 'f24' in x['crew']))]
#     avg = (temp.loc[df_sailors['Seasons'].apply(lambda x: 'f24' in x['skipper']), 'SkipperOrdinal'].mean() + temp.loc[df_sailors['Seasons'].apply(lambda x: 'f24' in x['crew']), 'CrewOrdinal'].mean()) / 2
#     avgRatio = (temp.loc[df_sailors['Seasons'].apply(lambda x: 'f24' in x['crew']),'skipperAvgRatio'].mean() + temp.loc[df_sailors['Seasons'].apply(lambda x: 'f24' in x['crew']),'crewAvgRatio'].mean()) / 2
#     numCurMembers = len(temp)
    
    region = teamRegions[team]
    # teamLink = df_races.loc[df_races['Team'] == team, 'Teamlink'].iloc[0]
    teamLink = team_link_map.get(team, None)  # Default to None if team not found
    url = f"https://scores.collegesailing.org/schools/{teamLink.split("/")[2]}"
    
    if scrape:
        page = requests.get(url)
        teamPage = BeautifulSoup(page.content, 'html.parser')
        
        try:
            region = teamPage.find('span', class_="page-info-value").contents[0].contents[0]
        except:
            print(url)
            continue
        
    filtered_people = [p for p in people.values() if team in p.teams]
    
    members = [{"name": p.name,
                "key": p.key,
                'teams': list(p.teams),
                'skipperRating': int(p.sr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                'crewRating': int(p.cr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                'avgSkipperRatio': float(p.avgSkipperRatio),
                'avgCrewRatio': float(p.avgCrewRatio),
                'raceCount': getCounts(p.races),
                'seasons':{'skipper': list(p.seasons['skipper']), 'crew': list(p.seasons['crew'])},
                'cross': sum([race['cross'] for race in p.races]),
                'skipperRank': int(p.skipperRank),
                'crewRank': int(p.crewRank)} for p in filtered_people]
    
    teamRating = 0
    if numCurMembers > 0:
        teamRating = sum([p['skipperRating'] * (p['raceCount']['f24']/ 5) for p in members if 'f24' in p['raceCount'].keys() and p['raceCount']['f24'] > 5]) / numCurMembers
    
    topRating = 0
    numTops = 6
    if numCurMembers > 0:
        topSkippers = sorted([p['skipperRating'] for p in members
                                  if p['skipperRank'] != 0
                                  and p['cross'] > 20
                                  and 'f24' in p['raceCount'].keys() 
                                  and p['raceCount']['f24'] > 5
                                  ], reverse=True)[:numTops]
        topSkipperSum = sum(topSkippers)
        
        topCrews = sorted([p['crewRating'] for p in members
                               if p['crewRank'] != 0
                               and p['cross'] > 20
                               and 'f24' in p['raceCount'].keys()
                               and p['raceCount']['f24'] > 5
                               ], reverse=True)[:numTops]
        topCrewsSum = sum(topCrews)
        
        topRating = (topSkippers + topCrews) / (numTops * 2)
    
    #'#1': ,'#2': ,
    teams.append({"name":team, "avg": avg, 'avgRatio': avgRatio, 'teamRating': teamRating, 'topRating': topRating, "region": region, "link": url, 'memberCount': numCurMembers})
    # col.document().set({"name":team, "avg": avg, 'avgRatio': avgRatio, "region": region, "link": url, 'members': members})
    # if i > 20:
    #     break
#     if i % 20 == 0: # commit every 20 documents
#             batch.commit()
# batch.commit()
# doc = db.collection('vars').document('eloTeams').set({"teams": teams})
teams


0/207 Hawaii
1/207 Brown
2/207 Southern Cal
3/207 Salve Regina
4/207 UC Santa Barbara
5/207 Cal Poly
6/207 Washington
7/207 Channel Islands
8/207 UC San Diego
9/207 British Columbia
10/207 UC Los Angeles
11/207 Westmont College
12/207 Arizona State
13/207 Texas A&M Galveston
14/207 Texas A&M
15/207 Tulane
16/207 Rice
17/207 Texas
18/207 Oklahoma State
19/207 Texas A&M C. Christ
20/207 Central Oklahoma
21/207 Notre Dame
22/207 Jacksonville
23/207 Florida
24/207 Tennessee
25/207 Rollins
26/207 North Carolina State
27/207 Georgia Tech
28/207 Auburn
29/207 Charleston
30/207 South Florida
31/207 Old Dominion
32/207 Eckerd
33/207 Florida State
34/207 U. Miami
35/207 UW Milwaukee
36/207 Stony Brook
37/207 Duke
38/207 Clemson
39/207 U South Carolina
40/207 UNC Wilmington
41/207 Georgia
42/207 Berkeley
43/207 CSU Long Beach
44/207 Monterey Bay
45/207 UC Irvine
46/207 UC Davis
47/207 Rhode Island
48/207 Georgetown
49/207 Dartmouth
50/207 MIT
51/207 George Washington
52/207 Navy
53/207 Fordham
54

[{'name': 'Hawaii',
  'avg': np.float64(1379.6579561311885),
  'avgRatio': np.float64(0.6636753027963355),
  'teamRating': np.float64(6684.527272727273),
  'topRating': 1472.8333333333333,
  'region': 'PCCSC',
  'link': 'https://scores.collegesailing.org/schools/hawaii',
  'memberCount': np.float64(11.0)},
 {'name': 'Brown',
  'avg': np.float64(1364.9133803628629),
  'avgRatio': np.float64(0.6330551399629807),
  'teamRating': np.float64(10444.009523809524),
  'topRating': 1971.75,
  'region': 'NEISA',
  'link': 'https://scores.collegesailing.org/schools/brown',
  'memberCount': np.float64(42.0)},
 {'name': 'Southern Cal',
  'avg': np.float64(1197.3027996919982),
  'avgRatio': np.float64(0.5653393113978437),
  'teamRating': np.float64(7076.2),
  'topRating': 1436.8333333333333,
  'region': 'PCCSC',
  'link': 'https://scores.collegesailing.org/schools/southern-cal',
  'memberCount': np.float64(14.0)},
 {'name': 'Salve Regina',
  'avg': np.float64(1053.357969833743),
  'avgRatio': np.floa

In [40]:
topSkippers = []
for p in sorted([p for p in people.values() if p.skipperRank <= 100 and p.skipperRank != 0 and 'f24' in p.seasons['skipper']],key=lambda p: p.skipperRank):
    topSkippers.append({'name': p.name,'key': p.key, 'rank': int(p.skipperRank), 'team': list(p.teams), 'rating': p.sr.ordinal() + offset, 'seasons': list(p.seasons['skipper'])})
doc = db.collection('vars').document('topSkippers').set({"sailors": topSkippers})
topCrews = []
for p in sorted([p for p in people.values() if p.crewRank <= 100 and p.crewRank != 0 and 'f24' in p.seasons['crew']],key=lambda p: p.crewRank):
    topCrews.append({'name': p.name,'key': p.key, 'rank': int(p.crewRank), 'team': list(p.teams), 'rating': p.cr.ordinal() + offset, 'seasons': list(p.seasons['crew'])})
doc = db.collection('vars').document('topCrews').set({"sailors": topCrews})

In [39]:
import json
flattened_dict = {p.name: {'team': p.teams[-1], 'year': p.year[-1], 'key': p.key} for p in people.values() if 'f24' in p.seasons}
# print(flattened_dict)
doc = db.collection('vars').document('allSailors').set({'allSailors': json.dumps(flattened_dict, separators=(',', ':'))})

In [ ]:

for regatta in list(df_races_full['Regatta'].unique()):
    races = df_races_full[df_races_full['Regatta'] == regatta]
    raceIDs = list(races['raceID'].unique())
    links = races['Sailor'].unique()
    
    racePpl = [{"Name":p.name, 
         "Position": p.pos,
         "Teams": list(p.teams),
         "Rating": int(p.r.mu),
         "GlobalRank": int(p.rank),
         "races": [{'sailor': p.name,
                    'key': p.key,
                    'pos': p.pos,
                    "raceID": race['raceID'], 
                    "score": float(race['score']), 
                    "predicted": int(race['predicted']), 
                    "change": float(race['change']), 
                    'regAvg': float(race['regAvg']), 
                    'newRating': float(race['newRating']),
                    'date': race['date'],
                    'partner':race['partner'],
                    'ratio': float(race['ratio']),
                    'venue': race['venue']
                    } for race in p.races if race['raceID'].split("/")[0] + "/" +race['raceID'].split("/")[1] == regatta]
        } for p in people.values() if p.name in links]
    
    # race = {'raceID':'', 'raceNum':0, 'div': '', 'sailors':[]}
    # person = {'name':'', 'rating':0, 'change':0, 'team': '', 'pos': '', 'div':'', 'partner': ''}
    
    # for race in races['raceID'].unique():
    #     sailors = races[races['raceID'] == race, 'Sailor'].unique()
    # for p in [p for p in people.values() if p.name in sailors]:
        # racePpl.append({'name':p.name, 'rating':p.rating, 'changes':p.changes, 'team': p.team, 'pos': '', 'div':'', 'partner': ''})
    # print(regatta)
    db.collection('eloRegattas').document().set({'regattaName': regatta,'raceIDs':raceIDs, 'sailors': racePpl}, timeout=15)

In [ ]:
def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        if deleted % 50 == 0:
            print(f"{deleted} Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)
col = db.collection('eloRegattas')
delete_collection(col, 400)